Floyd–Warshall AND Edmonds–Karp


In [17]:
import random
import time
import statistics
from collections import deque

INF = float('inf')

def floyd_warshall(n, edges):
    dist = [[INF]*n for _ in range(n)]
    for i in range(n):
        dist[i][i] = 0.0
    for u,v,w in edges:
        if w < dist[u][v]:
            dist[u][v] = float(w)
    for k in range(n):
        dk = dist[k]
        for i in range(n):
            di = dist[i]
            dik = di[k]
            if dik == INF:
                continue
            for j in range(n):
                alt = dik + dk[j]
                if alt < di[j]:
                    di[j] = alt
    neg_cycle = any(dist[i][i] < 0 for i in range(n))
    return dist, neg_cycle

def edmonds_karp_max_flow(n, edge_list, s, t):
    adj = [[] for _ in range(n)]
    cap = {}
    for u,v,w in edge_list:
        if (u,v) not in cap:
            adj[u].append(v)
            adj[v].append(u)
            cap[(u,v)] = int(w)
            cap[(v,u)] = 0
        else:
            cap[(u,v)] += int(w)
    aug_count = 0
    flow = 0
    parent = [-1]*n
    while True:
        for i in range(n):
            parent[i] = -1
        parent[s] = -2
        q = deque()
        q.append((s, float('inf')))
        pushed = 0
        while q:
            u, f = q.popleft()
            for v in adj[u]:
                if parent[v] == -1 and cap.get((u,v),0) > 0:
                    parent[v] = u
                    nf = f if f < cap[(u,v)] else cap[(u,v)]
                    if v == t:
                        pushed = nf
                        break
                    q.append((v,nf))
            if pushed:
                break
        if pushed == 0:
            break
        aug_count += 1
        flow += pushed
        v = t
        while v != s:
            u = parent[v]
            cap[(u,v)] -= pushed
            cap[(v,u)] = cap.get((v,u),0) + pushed
            v = u
    return flow, aug_count, adj, cap

def gen_dense_weighted_graph(n, density=1.0, weight_range=(1,100), seed=None):
    if seed is not None:
        random.seed(seed)
    edges = []
    for u in range(n):
        for v in range(n):
            if u == v:
                continue
            if random.random() <= density:
                w = random.randint(weight_range[0], weight_range[1])
                edges.append((u,v,w))
    return edges

def gen_random_flow_edges(n, m, cap_range=(1,20), seed=None):
    if seed is not None:
        random.seed(seed)
    edges = []
    tries = 0
    while len(edges) < m and tries < m*10:
        u = random.randrange(n)
        v = random.randrange(n)
        if u==v:
            tries += 1
            continue
        c = random.randint(cap_range[0], cap_range[1])
        edges.append((u,v,c))
    return edges

def format_time(x):
    if x >= 1.0:
        return f"{x:.2f}s"
    if x >= 0.001:
        return f"{x*1000:.1f}ms"
    return f"{x*1e6:.0f}μs"

def estimate_fw_memory_bytes(n):
    base = n*n*8
    py_overhead_factor = 8
    return int(base * py_overhead_factor)

def estimate_ek_memory_bytes(n, m):
    entries = 2*m
    base = entries * 8
    py_overhead_factor = 6
    adj_base = entries * 8
    return int((base + adj_base) * py_overhead_factor)

def run_fw_experiments(sizes, trials=3, seed=0):
    rows = []
    for n in sizes:
        times = []
        for t in range(trials):
            edges = gen_dense_weighted_graph(n, density=1.0, weight_range=(1,100), seed=seed+t)
            t0 = time.perf_counter()
            _, neg = floyd_warshall(n, edges)
            t1 = time.perf_counter()
            times.append(t1-t0)
        mean = statistics.mean(times)
        sd = statistics.stdev(times) if len(times)>1 else 0.0
        mem = estimate_fw_memory_bytes(n)
        rows.append(("Floyd–Warshall", n, "-", mean, sd, mem, neg))
    return rows

def run_ek_experiments(n, m_values, trials=3, seed=0):
    rows = []
    for m in m_values:
        times = []
        flows = []
        augs = []
        mems = []
        for t in range(trials):
            edges = gen_random_flow_edges(n, m, cap_range=(1,20), seed=seed+t)
            t0 = time.perf_counter()
            flow, aug, adj, cap = edmonds_karp_max_flow(n, edges, 0, n-1)
            t1 = time.perf_counter()
            times.append(t1-t0)
            flows.append(flow)
            augs.append(aug)
            mems.append(estimate_ek_memory_bytes(n,m))
        mean_t = statistics.mean(times)
        sd_t = statistics.stdev(times) if len(times)>1 else 0.0
        mean_flow = statistics.mean(flows)
        mean_aug = statistics.mean(augs)
        mean_mem = int(statistics.mean(mems))
        rows.append(("Edmonds–Karp", n, m, mean_t, sd_t, mean_flow, mean_aug, mean_mem))
    return rows

def print_fw_table(rows):
    print("### Floyd–Warshall (APSP) results")
    print()
    print("| Algorithm | n | m | mean time | stdev | mem estimate | negative cycle |")
    print("|---|---:|:---:|:---:|:---:|:---:|:---:|")
    for alg,n,m,mean,sd,mem,neg in rows:
        mt = format_time(mean)
        sd_s = format_time(sd)
        mem_kb = f"{mem//1024:,} KB"
        neg_s = "yes" if neg else "no"
        print(f"| {alg} | {n} | {m} | {mt} | {sd_s} | {mem_kb} | {neg_s} |")

def print_ek_table(rows):
    print("### Edmonds–Karp (Maximum Flow) results")
    print()
    print("| Algorithm | n | m | mean time | stdev | mean flow | mean augments | mem estimate |")
    print("|---|---:|---:|:---:|:---:|:---:|:---:|:---:|")
    for alg,n,m,mt,sd,flow,aug,mem in rows:
        mt_s = format_time(mt)
        sd_s = format_time(sd)
        mem_kb = f"{mem//1024:,} KB"
        print(f"| {alg} | {n} | {m} | {mt_s} | {sd_s} | {int(flow)} | {aug:.1f} | {mem_kb} |")

if __name__ == "__main__":
    fw_sizes = [50, 100, 200]
    fw_rows = run_fw_experiments(fw_sizes, trials=3, seed=1)
    ek_n = 50
    ek_m_values = [50, 100, 250]
    ek_rows = run_ek_experiments(ek_n, ek_m_values, trials=3, seed=2)
    print_fw_table(fw_rows)
    print()
    print_ek_table(ek_rows)


### Floyd–Warshall (APSP) results

| Algorithm | n | m | mean time | stdev | mem estimate | negative cycle |
|---|---:|:---:|:---:|:---:|:---:|:---:|
| Floyd–Warshall | 50 | - | 23.1ms | 2.6ms | 156 KB | no |
| Floyd–Warshall | 100 | - | 224.6ms | 56.3ms | 625 KB | no |
| Floyd–Warshall | 200 | - | 645.7ms | 408.8ms | 2,500 KB | no |

### Edmonds–Karp (Maximum Flow) results

| Algorithm | n | m | mean time | stdev | mean flow | mean augments | mem estimate |
|---|---:|---:|:---:|:---:|:---:|:---:|:---:|
| Edmonds–Karp | 50 | 50 | 44μs | 7μs | 0 | 0.0 | 9 KB |
| Edmonds–Karp | 50 | 100 | 111μs | 48μs | 6 | 2.3 | 18 KB |
| Edmonds–Karp | 50 | 250 | 321μs | 88μs | 23 | 7.7 | 46 KB |


Floyd–Warshall times grow rapidly with n because the algorithm performs three nested loops over all vertices, so runtime scales on the order of n³; the measurements (≈23 ms → 225 ms → 646 ms as n doubles) reflect that cubic growth plus Python loop overhead, and the memory estimates increase quadratically because the distance matrix requires O(n²) storage. Edmonds–Karp runs are much faster here because the tested networks are small and sparse and produced few or no s→t augmenting paths, so BFS searches terminate quickly and the algorithm performs only a small number of augmentations (hence very small mean flow and augment counts); measured microsecond-level times show that for these input sizes the per-augmentation BFS cost is negligible in Python. Variation in stdev values comes from random graph structure and OS scheduling; overall the tables align with theory: FW is computation- and memory-dominant for dense APSP, while EK’s cost depends on connectivity and number of augmentations, making it cheap on lightly connected small graphs but potentially expensive on dense or adversarial instances